# This notebook generates the data defining the background traffic

A .csv file is created containing:
 - Origin
 - Destination
 - Departure time
 - Type of drone
for each drone forming the background traffic.

In [1]:
"""
We define a default path where the modules developed in USEPE are stored
"""

default_path = r"C:\workspace3\bluesky"

"""
We define a second default path where the scenarios are stored
"""

default_path2 = r"C:\workspace3\scenarios-exercise1\scenario\USEPE\exercise_1"


In [2]:
import os
import sys
import configparser
import pickle
import osmnx as ox
import random
import pandas as pd
import datetime

os.chdir(default_path)

c:\workspace3\python3_env\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


Configuration file¶

We create a config file with the following fields:

[City]

mode = rectangle

#hannover_lat = 52.376

#hannover_lon = 9.76

#zone_size = 1000

hannover_lat_min = 52.297839999994714

hannover_lat_max = 52.438260000005286

hannover_lon_min = 9.643080000001035

hannover_lon_max = 9.844349999998968

import = True

imported_graph_path = ./data/exercise_1_reference.graphml

[Layers]
number_of_layers = 9

layer_width = 25

[BuildingData]

lat_min = 52.297839999994714

lat_max = 52.438260000005286

lon_min = 9.643080000001035

lon_max = 9.844349999998968

divisions = 20

directory_hannover = C:\workspace3\bluesky_usepe_git\usepe\city_model\data\CityGML_LoD1

[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2

[Outputs]

graph_path = ./data/exercise_1_reference.graphml

[Segments]

import = True

path = ./data/reference_segments.pkl

[Corridors]

corridors =

altitude = 250

delta_z = 25

speed = 50

acceleration_length = 50

file_path_corridors = ./data/usepe-hannover-corridors.geojson

[Strategic_Deconfliction]

ratio = 3

delay = 60

In [3]:
 # -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = default_path2 + r"\settings_exercise_1_reference.cfg"
config = configparser.ConfigParser()
config.read( config_path )

['C:\\workspace3\\scenarios-exercise1\\scenario\\USEPE\\exercise_1\\settings_exercise_1_reference.cfg']

In [4]:
def backgroudTraffic(density, avg_flight_duration, simulation_time, config ):
    '''
    This function creates distributed origins and destinations for the background traffic in 
    the city area defined in the configuration file
    
    Input:
        density - density desired
        avg_flight_duration - duration of the flights in average 
        simulation time - duration of the simulation (seconds)
        config - configuration object
    
    Output:
        csv file 
    '''
    mode = config['City'].get('mode')
    
    if mode == 'rectangle':
        lat_min = config['City'].getfloat( 'hannover_lat_min' )
        lat_max = config['City'].getfloat( 'hannover_lat_max' )
        lon_min = config['City'].getfloat( 'hannover_lon_min' )
        lon_max = config['City'].getfloat( 'hannover_lon_max' )
        
        latitude_distance = ox.distance.great_circle_vec(lat_min, lon_min, lat_max, lon_min)
        longitude_distance = ox.distance.great_circle_vec(lat_min, lon_min, lat_min, lon_max)
    
        area = (latitude_distance / 1000) * (longitude_distance / 1000)
    
    elif mode == 'square':
        zone_size = config['City'].getint('zone_size')
        
        area = (zone_size/1000)**2
    
    flights_second = density * area / avg_flight_duration
    
    time_spacing = 1 / flights_second
    
    total_flights = flights_second * simulation_time
    
    # Drone flight plan
    min_flight_distance = 1000
    max_flight_distance = 20000
    # Consider that the background traffic includes different types of drones
    drone_type_distribution = { "M600": 0.5,
                                "Amzn": 0.3,
                                "W178": 0.2 }
    n = 1
    data = {'origin': [], 'destination': [], 'departure': [], 'drone': []}
    while n <= total_flights:
        orig_lat = random.uniform( lat_min, lat_max )
        orig_lon = random.uniform( lon_min, lon_max )
        dest_lat = random.uniform( lat_min, lat_max )
        dest_lon = random.uniform( lon_min, lon_max )
        
        if ox.distance.great_circle_vec( orig_lon, orig_lat, dest_lon, dest_lat ) < min_flight_distance:
            # We discard the trajectory if the origin and destination are too close
            continue
        elif ox.distance.great_circle_vec( orig_lon, orig_lat, dest_lon, dest_lat ) > max_flight_distance:
            # We discard the trajectory if the origin and destination are too far away
            continue
        
        orig = (orig_lon, orig_lat)
        dest = (dest_lon, dest_lat)
        
        drone_type = random.choices( list(drone_type_distribution.keys()), 
                                     weights=tuple(drone_type_distribution.values()))[0]
        
        departure_time = '{}'.format( n * time_spacing )
        if n * time_spacing < 36000:
            departure_time = '0{}'.format( str( datetime.timedelta( seconds=n * time_spacing ) ) )
        else:
            departure_time = str( datetime.timedelta( seconds=n * time_spacing ) )
        departure_time = departure_time[:11]
        
        data['origin'].append(orig)
        data['destination'].append(dest)
        data['departure'].append(departure_time)
        data['drone'].append(drone_type)
        
        n += 1
    
    data_frame = pd.DataFrame(data)
    
    file_name = 'background_traffic_' + str(density).replace('.', '-') + '_' + str(int(avg_flight_duration)) + '_' + str(simulation_time) + '.csv'
    
    path = sys.path[0] + '\\data\\' + file_name
    
    data_frame.to_csv(path, sep=';')
    
    return data_frame, path

# Data for the experiments

The densities to be tested in the experiments are
 - Low: 0.0964 simultaneous flights / km^2
 - Medium: 0.964 simultaneous flights / km^2
 - High: 9.64 simultaneous flights / km^2
 
The average flight duration calculated is: 2384 s

The simulation time considered is: 2h (7200 s)

In [6]:
density = {'Low': 0.0964, 'Medium': 0.964, 'High': 9.64}

avg_flight_duration = 2384

simulation_time = 7200

data_frame, path = backgroudTraffic(density['High'], avg_flight_duration, simulation_time, config )

In [7]:
data_frame

,origin,destination,departure,drone
0,"(9.737156280632169, 52.39437777698964)","(9.677488473494854, 52.32859244097638)",00:00:01.15,W178
1,"(9.753033336978177, 52.37243412726987)","(9.680635848494116, 52.37410943358341)",00:00:02.31,Amzn
2,"(9.702073305937654, 52.391873200386065)","(9.719332746150355, 52.329977781323315)",00:00:03.47,Amzn
3,"(9.705744700635858, 52.3119187036991)","(9.686532942940687, 52.419030224858496)",00:00:04.62,Amzn
4,"(9.659005899921336, 52.345103195193005)","(9.729176785718535, 52.41326796105709)",00:00:05.78,Amzn
...,...,...,...,...
6216,"(9.662920646586967, 52.41969534441477)","(9.747026856012035, 52.33623025816888)",01:59:54.39,Amzn
6217,"(9.776079572760331, 52.410995844081775)","(9.793531117386541, 52.41681738636014)",01:59:55.55,M600
6218,"(9.686356385270658, 52.373004711236305)","(9.724096393696266, 52.417626836517286)",01:59:56.70,M600
6219,"(9.75653255834739, 52.30167610373276)","(9.758758468224007, 52.416482464703634)",01:59:57.86,M600


In [ ]:
path